In [9]:
import numpy as np
import scipy
from sklearn.metrics.pairwise import euclidean_distances


def cov_RBF(x1, x2, parameters):  
    
    lengthscale_square =   parameters[:,0]  # lengthscale^2
    variance = parameters[:,1] # sigma^2


    if x1.shape[1]!=x2.shape[1]:
        x1=np.reshape(x1,(-1,x2.shape[1]))

    Euc_dist=euclidean_distances(x1,x2)

    return variance*np.exp(-0.5*np.square(Euc_dist)/lengthscale_square)

def log_llk(X,y,parameters):

    noise_delta = 10**(-6)
    
    parameters = parameters.reshape(-1,2)
    
    if np.isnan(parameters).any():
        print('issue with scipy.minimize!')
        
        return -np.inf

    KK_x_x=cov_RBF(X,X,parameters)+np.eye(len(X))*noise_delta     
    if np.isnan(KK_x_x).any(): #NaN
        print("nan in KK_x_x !")   
        #print('X is: ',X)
        print('parameter is: ',parameters)
        print(np.isnan(parameters).any())

    try:
        L=scipy.linalg.cholesky(KK_x_x,lower=True)
        alpha=np.linalg.solve(KK_x_x,y)

    except: # singular
        return -np.inf
    
    try:
        first_term = -0.5*np.log(np.linalg.det(KK_x_x))
        
        KK_inv = np.linalg.inv(KK_x_x)
        second_term = -0.5* np.dot(np.dot(y.T,KK_inv),y)
            

    except: # singular
        return -np.inf

    logmarginal = first_term+second_term -0.5*len(y)*np.log(2*3.1415926)
    
    return logmarginal.item()


In [7]:
X = np.array([[1,1],[1.5,0.8],[2.1,0.3],[1.35,2.0]])

In [8]:
euclidean_distances(X,X)

array([[0.        , 0.53851648, 1.30384048, 1.05948101],
       [0.53851648, 0.        , 0.78102497, 1.20933866],
       [1.30384048, 0.78102497, 0.        , 1.85809042],
       [1.05948101, 1.20933866, 1.85809042, 0.        ]])

In [5]:
cov_RBF(X, X, np.array([[0.8**2,1.2**2]]))

array([[1.44      , 1.1480681 , 0.38156201, 0.59911003],
       [1.1480681 , 1.44      , 0.89411633, 0.45935359],
       [0.38156201, 0.89411633, 1.44      , 0.09704092],
       [0.59911003, 0.45935359, 0.09704092, 1.44      ]])

In [20]:
X = np.array([[1,1],[1.5,0.8],[2.1,0.3],[1.35,2.0]])
Y = np.array([0.8,1.2,0.3,0.75]).reshape(-1,1)

In [21]:
log_llk(X,Y,np.array([[0.8**2,1.2**2]]))

-4.236568433836075

In [18]:
parameters = np.array([[0.8**2,1.2**2]])
noise_delta = 10**(-6)
KK_x_x=cov_RBF(X,X,parameters)+np.eye(len(X))*noise_delta  

In [19]:
first_term = -0.5*np.log(np.linalg.det(KK_x_x))
print(first_term)
KK_inv = np.linalg.inv(KK_x_x)
print(KK_inv)
second_term = -0.5* np.dot(np.dot(Y.T,KK_inv),Y)
print(second_term)

0.250230431541768
[[ 2.63437602 -2.55876088  0.91373481 -0.34137112]
 [-2.55876088  3.77778011 -1.66576656 -0.02827084]
 [ 0.91373481 -1.66576656  1.48317113  0.05126376]
 [-0.34137112 -0.02827084  0.05126376  0.84203448]]
[[-0.81104477]]


In [22]:
first_term+second_term -0.5*len(Y)*np.log(2*3.1415926)

array([[-4.23656843]])

In [23]:
0.5*len(Y)*np.log(2*3.1415926)

3.675754098702369

In [24]:
np.log(2*3.1415926)

1.8378770493511845